In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

pandas    : 1.3.0
numpy     : 1.21.0
seaborn   : 0.11.1
matplotlib: 3.4.2



In [3]:
from scipy import interpolate

# https://github.com/DaniRuizPerez/PALM-Public-Respository/blob/master/Alignment/getAlignmentsIBD_Taxa.py

#Use B-spline to extrapolate values. NOTE: Parameters s must be adjusted appropriately to avoid over-fitting.
# tck = interpolate.splrep(timepoints, relativeAbundances, k=3, s=0.001, xb=weekFirstSample, xe=weekLastSample)

In [9]:
df_tax_splines = pd.read_csv("../results/tax_clr_splines.csv", index_col=0)

In [275]:
# PARAMETERS
PRESENCE_THRESHOLD = .95
SAMPLING_RATE = 1
OVERLAP_THRESHOLD = .95

max_study_day_no = df_tax_splines.index.max()
min_study_day_no = df_tax_splines.index.min()

a_values = np.arange(0.01, 3.01, .01)
b_values = np.arange(-max_study_day_no, max_study_day_no + .5, .5)

index_splines = np.arange(min_study_day_no, max_study_day_no, SAMPLING_RATE, dtype="int")

In [276]:
def linear_warp(a, b, s):
	return (s - b) / a

In [277]:
lb_linear_constraint = (index_splines.max() - index_splines.min()) * OVERLAP_THRESHOLD

In [278]:
def inv_linear_warp(a, b, s):
	return (a*s) + b

In [279]:
usernames = df_tax_splines["UserName"].unique()

In [280]:
df_tax_counts = pd.read_csv("../data/taxonomy_counts_s_t20.txt", index_col=0, sep="\t")
df_mapping = pd.read_csv("../data/SampleID_map.txt", sep='\t', index_col=0)

In [281]:
df_tax_counts = df_tax_counts.T

df_tax_counts.index.name = "#SampleID"

df_tax_counts_username = pd.merge(df_tax_counts, df_mapping.reset_index()[["#SampleID", "UserName"]], on="#SampleID", how="left")

df_tax_counts_username = df_tax_counts_username.set_index("#SampleID")

df_username_presence = df_tax_counts_username.groupby("UserName").apply(lambda x: (x > 0).sum() / x.shape[0])

In [282]:
import pickle
with open("../results/d_splines.pkl", "rb") as inf:
    d_splines = pickle.load(inf)

In [283]:
a = 1.5
b = -5

# usernames
# reference
# PRESENCE_THRESHOLD
# d_splines

def error_a_b_reference(ab, reference, usernames, presence_threshold, d_splines, overlap_threshold):
    a = ab[0]
    b = ab[1]
    reference_inv_warp = inv_linear_warp(a, b, index_splines)

    # alpha is the max(reference_sample, inv_linear_warp(current_sample))
    # beta is the min(reference_sample_t, inv_linear_warp(current_sample))
    alpha = max(index_splines.min(), reference_inv_warp.min())
    beta = min(index_splines.max(), reference_inv_warp.max())

    overlap =  (beta - alpha) / (index_splines.max() - index_splines.min())

#     if overlap > overlap_threshold and alpha < beta:
    error = get_alignment_error(a, b, alpha, beta, index_splines, reference, usernames, presence_threshold, d_splines)
#     else:
#         error = get_alignment_error(1, 0, index_splines.min(), index_splines.max(), index_splines, reference, usernames, presence_threshold, d_splines)
    return error

In [301]:
def get_alignment_error(a, b, alpha, beta, index_splines, reference, usernames, presence_threshold, d_splines):
    current_warp = linear_warp(a, b, index_splines)
    reference_taxonomies = set(df_username_presence.loc[reference, df_username_presence.loc[reference, :] > PRESENCE_THRESHOLD].index)
    errors = np.repeat(0.0, len(usernames) -1)
    ix = 0
    for current in usernames:
        if reference != current:
            current_taxonomies = set(df_username_presence.loc[current, df_username_presence.loc[reference, :] > PRESENCE_THRESHOLD].index)
            filtered_taxonomies = reference_taxonomies.intersection(current_taxonomies)
            error_tax = np.repeat(0.0, len(filtered_taxonomies))
            for ix_tax, taxonomy in enumerate(filtered_taxonomies):
                spline_reference, reference_min, reference_max = d_splines[reference][taxonomy]
                spline_current, current_min, current_max = d_splines[current][taxonomy]
                
                print(index_splines)
                print(current_warp)
                
                ts_reference = interpolate.splev(index_splines, spline_reference)
                ts_current_warp = interpolate.splev(current_warp, spline_current)

                ts_reference = np.clip(ts_reference, reference_min, reference_max)
                ts_current_warp = np.clip(ts_current_warp, current_min, current_max)

                cur_error = np.linalg.norm(ts_reference - ts_current_warp)
                cur_error = cur_error / (beta - alpha)
                error_tax[ix_tax] = cur_error
            errors[ix] = np.mean(error_tax)
            ix += 1
    return np.mean(errors)

In [302]:
ts_reference

array([ 9.6053645 ,  8.89735334,  9.40860635,  9.05187412, 10.10132845,
        8.35122827,  9.8441164 ,  8.50981637,  9.56265569,  9.20640963,
        9.15059699,  9.1326518 ,  8.97162731,  7.96790482,  7.39256677])

In [300]:
ts_current_warp

array([6.56190963, 6.56493858, 6.70964954, 6.74805422, 6.16878747,
       5.95981741, 6.20934823, 6.06607585, 6.01915342, 6.25694923,
       6.40271912, 6.41706202, 6.3032529 , 6.12081157, 6.31155251])

In [307]:
error_a_b_reference((3.        , -4.59736804), reference, usernames, PRESENCE_THRESHOLD, d_splines, OVERLAP_THRESHOLD)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[1.86578935 2.19912268 2.53245601 2.86578935 3.19912268 3.53245601
 3.86578935 4.19912268 4.53245601 4.86578935 5.19912268 5.53245601
 5.86578935 6.19912268 6.53245601]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[1.86578935 2.19912268 2.53245601 2.86578935 3.19912268 3.53245601
 3.86578935 4.19912268 4.53245601 4.86578935 5.19912268 5.53245601
 5.86578935 6.19912268 6.53245601]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[1.86578935 2.19912268 2.53245601 2.86578935 3.19912268 3.53245601
 3.86578935 4.19912268 4.53245601 4.86578935 5.19912268 5.53245601
 5.86578935 6.19912268 6.53245601]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[1.86578935 2.19912268 2.53245601 2.86578935 3.19912268 3.53245601
 3.86578935 4.19912268 4.53245601 4.86578935 5.19912268 5.53245601
 5.86578935 6.19912268 6.53245601]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[1.86578935 2.19912268 2.53245601 2.86578935 3.19912268 3.53245601
 3.86578935 4.19912268

0.7766121512591742

In [286]:
from scipy.optimize import minimize

In [287]:
usernames

array(['MCTs01', 'MCTs03', 'MCTs04', 'MCTs05', 'MCTs06', 'MCTs07',
       'MCTs08', 'MCTs09', 'MCTs10', 'MCTs11', 'MCTs12', 'MCTs13',
       'MCTs14', 'MCTs15', 'MCTs16', 'MCTs18', 'MCTs19', 'MCTs20',
       'MCTs21', 'MCTs22', 'MCTs23', 'MCTs24', 'MCTs25', 'MCTs26',
       'MCTs27', 'MCTs28', 'MCTs29', 'MCTs31', 'MCTs32', 'MCTs33',
       'MCTs34', 'MCTs35', 'MCTs36', 'MCTs37'], dtype=object)

In [288]:
def gen_constraint_overlap(index_splines):
    def inner_constraint(ab):
        a = ab[0]
        b = ab[1]
        reference_inv_warp = inv_linear_warp(a, b, index_splines)
        reference_inv_warp = inv_linear_warp(a, b, index_splines)

        # alpha is the max(reference_sample, inv_linear_warp(current_sample))
        # beta is the min(reference_sample_t, inv_linear_warp(current_sample))
        alpha = max(index_splines.min(), reference_inv_warp.min())
        beta = min(index_splines.max(), reference_inv_warp.max())

        overlap =  (beta - alpha) / (index_splines.max() - index_splines.min())
        return overlap
    return inner_constraint
    
def gen_constraint_greater_than(index_splines):
    def inner_constraint(ab):
        a = ab[0]
        b = ab[1]
        reference_inv_warp = inv_linear_warp(a, b, index_splines)
        reference_inv_warp = inv_linear_warp(a, b, index_splines)

        # alpha is the max(reference_sample, inv_linear_warp(current_sample))
        # beta is the min(reference_sample_t, inv_linear_warp(current_sample))
        alpha = max(index_splines.min(), reference_inv_warp.min())
        beta = min(index_splines.max(), reference_inv_warp.max())

        return beta - alpha
    return inner_constraint

In [296]:
bounds = [(0.01, 3), (-15, 15)]

from scipy.optimize import NonlinearConstraint

constraint_overlap = NonlinearConstraint(gen_constraint_overlap(index_splines), lb=OVERLAP_THRESHOLD, ub=np.inf)
constraint_gt = NonlinearConstraint(gen_constraint_greater_than(index_splines), lb=0, ub=np.inf)
constraint_cobyla_a = LinearConstraint((1,0), lb=0.01, ub=3)
constraint_cobyla_b = LinearConstraint((0,1), lb=-15, ub=15)

# reference = usernames[0]

results = []

for reference in usernames:
    res = minimize(
        error_a_b_reference,
    #     method="COBYLA",
        x0=[1, 0.0],
        args=(reference, usernames, PRESENCE_THRESHOLD, d_splines, OVERLAP_THRESHOLD),
        constraints=(constraint_overlap, constraint_gt, constraint_cobyla_a, constraint_cobyla_b),
    #     bounds=bounds,
    )
    results.append((reference, res))

In [298]:
results

[('MCTs01',
       fun: 0.32952855302659506
       jac: array([-8.95962119e-04, -9.01930034e-05])
   message: 'Optimization terminated successfully'
      nfev: 63
       nit: 19
      njev: 19
    status: 0
   success: True
         x: array([  2.76719325, -14.74703799])),
 ('MCTs03',
       fun: 0.33793811024375703
       jac: array([8.12049955e-04, 8.88146460e-05])
   message: 'Optimization terminated successfully'
      nfev: 67
       nit: 20
      njev: 20
    status: 0
   success: True
         x: array([ 1.14195089, -1.44026918])),
 ('MCTs04',
       fun: 0.3911035132079273
       jac: array([0.00224509, 0.00031962])
   message: 'Optimization terminated successfully'
      nfev: 42
       nit: 13
      njev: 13
    status: 0
   success: True
         x: array([ 1.60053914, -3.46733409])),
 ('MCTs05',
       fun: 0.3973193033539714
       jac: array([0.00043601, 0.00015108])
   message: 'Optimization terminated successfully'
      nfev: 73
       nit: 23
      njev: 23
    statu

In [102]:
# UserName to Set of Taxonomy above threshold
present_taxa = dict()

In [ ]:
# 			T = [warpFunction(a, b, timepointReferenceSample.offsetID) for timepointReferenceSample in timepointsListReferenceSample]
# ##			T_inverse = [warpFunctionInverse(a, b, timepointCurrentSample.offsetID) for timepointCurrentSample in timepointsListCurrentSample]
# 			timepointCurrentSampleMin = warpFunctionInverse(a, b, min(CurrentSampleT))
# 			timepointCurrentSampleMax = warpFunctionInverse(a, b, max(CurrentSampleT))
# 			alpha = max(timepointReferenceSampleMin, timepointCurrentSampleMin)
# 			beta = min(timepointReferenceSampleMax, timepointCurrentSampleMax)
# 			overlap =  (beta - alpha) / (timepointReferenceSampleMax - timepointReferenceSampleMin)
# 			if overlap > OVERLAP_THRESHOLD and alpha < beta:
# 				[alignmentError, a, b] = getAlignmnetError(a, b, alpha, beta, timepointsListReferenceSample, timepointsListCurrentSample, taxonWeights, useSplines)
# 				if len(optimalAlignmentParameters) == 0 or optimalAlignmentParameters[0] > alignmentError:
# 					optimalAlignmentParameters = [alignmentError, a, b, alpha, beta, overlap]